In [ ]:
import os

# 1. Paste your token between the quotes below
os.environ['KAGGLE_USERNAME'] = "enter your kaggle username url" # Look at your Kaggle profile URL to find this
os.environ['KAGGLE_KEY'] = "enter your kaggle key"

print("Credentials set!")

In [ ]:
# Create a folder for the data
!mkdir -p data

# Download the specific finance dataset
!kaggle datasets download -d aryan208/financial-transactions-dataset-for-fraud-detection

# Unzip it
import zipfile
with zipfile.ZipFile("financial-transactions-dataset-for-fraud-detection.zip", "r") as zip_ref:
    zip_ref.extractall("data")

print("Dataset is ready in the 'data' folder!")

In [ ]:
import os
import pandas as pd

# 1. Let's see what is actually inside the 'data' folder
files_in_data = os.listdir('data')
print(f"Files found in the data folder: {files_in_data}")

# 2. Automatically pick the first CSV file found
csv_file = [f for f in files_in_data if f.endswith('.csv')][0]
file_path = os.path.join('data', csv_file)

# 3. Load it
df = pd.read_csv(file_path)

print(f"\nSuccess! Loaded file: {csv_file}")
df.head()

In [ ]:
# Install the fuzzy matching tool
!pip install thefuzz

In [ ]:
# Look at the unique merchant names to see the mess
unique_merchants = df['Merchant'].unique()
print(f"Total unique merchant names found: {len(unique_merchants)}")
print("First 10 merchant names in your data:")
print(unique_merchants[:10])

In [ ]:
# This lists all the actual column names in your table
print("The columns in this dataset are:")
print(df.columns.tolist())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Group the data by category and sum the amounts
category_spend = df.groupby('merchant_category')['amount'].sum().sort_values(ascending=False)

# 2. Create a bar chart
plt.figure(figsize=(12,6))
category_spend.plot(kind='bar', color='skyblue')
plt.title('Total Spending by Merchant Category')
plt.ylabel('Total Amount ($)')
plt.xticks(rotation=45)
plt.show()

print("This chart shows us where the bulk of the money is going!")

In [ ]:
# 1. Calculate the mean and standard deviation for the 'amount' column
mean_amt = df['amount'].mean()
std_amt = df['amount'].std()

# 2. Apply the Z-Score formula: (Value - Mean) / Standard Deviation
df['z_score'] = (df['amount'] - mean_amt) / std_amt

# 3. Flag anything with a Z-Score greater than 3 as an Anomaly
df['is_anomaly'] = df['z_score'].abs() > 3

# 4. Show us only the anomalies we found!
anomalies = df[df['is_anomaly'] == True]
print(f"Total Anomalies Found: {len(anomalies)}")
anomalies[['merchant_category', 'amount', 'z_score']].head()

In [ ]:
# 1. Calculate Z-Score specifically for each category
df['z_score_category'] = df.groupby('merchant_category')['amount'].transform(
    lambda x: (x - x.mean()) / x.std()
)

# 2. Flag anomalies again with the new specific score
df['is_anomaly_v2'] = df['z_score_category'].abs() > 3

# 3. Check the count again
refined_anomalies = df[df['is_anomaly_v2'] == True]
print(f"Total Refined Anomalies Found: {len(refined_anomalies)}")

# 4. Let's see the most 'extreme' ones
refined_anomalies[['merchant_category', 'amount', 'z_score_category']].sort_values(by='z_score_category', ascending=False).head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Let's look at one category to see why the math is flagging so much
sample_cat = df['merchant_category'].unique()[0]
subset = df[df['merchant_category'] == sample_cat]

plt.figure(figsize=(10,6))
sns.histplot(subset['amount'], bins=50, kde=True, color='purple')
plt.title(f'Distribution of Amounts for {sample_cat}')
plt.show()

print(f"Average amount for {sample_cat}: {subset['amount'].mean()}")
print(f"Wiggle room (Std Dev) for {sample_cat}: {subset['amount'].std()}")

In [ ]:
# 1. Define a function to find outliers using the IQR method
def find_iqr_outliers(group):
    Q1 = group.quantile(0.25)
    Q3 = group.quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    return (group > upper_bound)

# 2. Apply this logic to every category
df['is_outlier_iqr'] = df.groupby('merchant_category')['amount'].transform(find_iqr_outliers)

# 3. Count the new, smarter anomalies
iqr_anomalies = df[df['is_outlier_iqr'] == True]
print(f"Total Robust Anomalies Found: {len(iqr_anomalies)}")

# 4. Look at the biggest ones
iqr_anomalies[['merchant_category', 'amount', 'date']].sort_values(by='amount', ascending=False).head()

In [ ]:
# 1. Fix the KeyError: Make sure 'date' exists
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date

# 2. Define our "Human + Math" Filter
# We want items that the IQR flagged AND are above a certain dollar amount (e.g., 500)
# This removes the "pennies" that the math flagged as anomalies.
hard_threshold = 500

final_anomalies = df[
    (df['is_outlier_iqr'] == True) &
    (df['amount'] > hard_threshold)
]

print(f"Final Professional Anomalies: {len(final_anomalies)}")

# 3. View the results (Now including 'date' safely!)
final_anomalies[['date', 'merchant_category', 'amount']].sort_values(by='amount', ascending=False).head(10)

In [ ]:
# 1. Fix the ValueError: Use 'ISO8601' to handle different timestamp lengths
# We also add errors='coerce' just in case there's a totally broken piece of text
df['timestamp'] = pd.to_datetime(df['timestamp'], format='ISO8601', errors='coerce')

# Create the date column (dropping any rows that couldn't be converted)
df = df.dropna(subset=['timestamp'])
df['date'] = df['timestamp'].dt.date

# 2. Define our "Human + Math" Filter
# We only want outliers that are actually significant amounts
hard_threshold = 500

final_anomalies = df[
    (df['is_outlier_iqr'] == True) &
    (df['amount'] > hard_threshold)
]

print(f"Final Professional Anomalies: {len(final_anomalies)}")

# 3. View the results
if not final_anomalies.empty:
    print(final_anomalies[['date', 'merchant_category', 'amount']].sort_values(by='amount', ascending=False).head(10))
else:
    print("No anomalies found over $500. Try lowering the hard_threshold to 100.")

In [ ]:
# 1. Count total transactions per category
total_counts = df['merchant_category'].value_counts()

# 2. Count anomaly transactions per category
anomaly_counts = final_anomalies['merchant_category'].value_counts()

# 3. Combine them into a "Risk Table"
risk_report = pd.DataFrame({
    'Total_Transactions': total_counts,
    'Total_Anomalies': anomaly_counts
}).fillna(0) # Replace missing values with 0

# 4. Calculate the Risk Percentage (Density)
risk_report['Risk_Percentage'] = (risk_report['Total_Anomalies'] / risk_report['Total_Transactions']) * 100

# 5. Sort by highest risk
risk_report = risk_report.sort_values(by='Risk_Percentage', ascending=False)

print("--- MERCHANT RISK REPORT ---")
print(risk_report)

In [ ]:
# Calculate the 'Fence' for each category
stats = df.groupby('merchant_category')['amount'].agg([
    lambda x: x.quantile(0.25),
    lambda x: x.quantile(0.75)
]).rename(columns={'<lambda_0>': 'Q1', '<lambda_1>': 'Q3'})

# Calculate IQR and the Upper Fence
stats['IQR'] = stats['Q3'] - stats['Q1']
stats['Upper_Fence'] = stats['Q3'] + (1.5 * stats['IQR'])

print("--- THE ANOMALY FENCES ---")
print(stats[['Q1', 'Q3', 'Upper_Fence']].sort_values(by='Upper_Fence', ascending=False))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the style
plt.figure(figsize=(12, 8))
sns.set_theme(style="whitegrid")

# Create a Boxplot of the top 5 most common categories to keep it clean
top_categories = df['merchant_category'].value_counts().nlargest(5).index
df_plot = df[df['merchant_category'].isin(top_categories)]

# Plotting
sns.boxplot(x='merchant_category', y='amount', data=df_plot, palette="Set2")

# Setting labels
plt.title('Spending Distribution & Anomaly Fences by Category', fontsize=16)
plt.ylabel('Transaction Amount ($)')
plt.xlabel('Merchant Category')
plt.yscale('log') # Using log scale because your data has a 'Long Tail'
plt.show()

In [ ]:
# Create a 'FacetGrid' to see each category clearly on its own scale
g = sns.FacetGrid(df_plot, col="merchant_category", sharey=False, height=5, aspect=0.8)
g.map(sns.boxplot, "amount", order=None, palette="Set2")

plt.subplots_adjust(top=0.8)
g.fig.suptitle('Zoomed-In View: Individual Category Distributions', fontsize=16)
plt.show()

In [ ]:
# Create the high-value anomaly list
final_report = final_anomalies[['date', 'merchant_category', 'amount']].sort_values(by='amount', ascending=False)

# Save to a CSV file you can download
final_report.to_csv('High_Value_Anomalies.csv', index=False)

print(f"Success! We found {len(final_report)} professional anomalies.")
print("The file 'High_Value_Anomalies.csv' is ready to download from your Colab files tab.")

In [ ]:
# Instead of user_id, let's see which CATEGORY is the 'leakies'
watchlist = final_anomalies['merchant_category'].value_counts()

print("--- TOP RISK CATEGORIES ---")
print(watchlist)

# Let's calculate the average 'Damage' ($) per category
damage_report = final_anomalies.groupby('merchant_category')['amount'].sum().sort_values(ascending=False)
print("\n--- TOTAL DOLLAR RISK PER CATEGORY ---")
print(damage_report)

In [ ]:
# Create the Z-Score feature so the AI knows 'relative' risk
df['z_score_category'] = df.groupby('merchant_category')['amount'].transform(
    lambda x: (x - x.mean()) / x.std()
).fillna(0) # Fill empty spots so the AI doesn't crash

In [ ]:
from sklearn.ensemble import IsolationForest

# 1. Select the signals: Raw Amount and the Z-Score Context
X = df[['amount', 'z_score_category']]

# 2. Initialize the Forest
# We use 100 trees (n_estimators) to vote on what is 'weird'
iso_forest = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)

# 3. Fit and Predict
# It learns the patterns and labels them: 1 (Normal) or -1 (Anomaly)
df['ml_anomaly_flag'] = iso_forest.fit_predict(X)

print("ML Training Complete.")

In [ ]:
# Filter out the AI-detected anomalies
ml_anomalies = df[df['ml_anomaly_flag'] == -1]

# Re-calculate your old statistical anomalies for comparison
stat_anomalies = df[df['z_score_category'].abs() > 3]

print(f"--- RESULTS ---")
print(f"Statistical Method (Z-Score > 3) found: {len(stat_anomalies)}")
print(f"AI Method (Isolation Forest) found: {len(ml_anomalies)}")

In [ ]:
# 1. Turn the 'timestamp' column into real dates/times
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])

# 2. Sort by User (cc_num) and Time so we can see their 'trail'
df = df.sort_values(by=['cc_num', 'trans_date_trans_time'])

# 3. THE MAGIC: Count how many times each card (cc_num) appeared in the last 24 hours
# We use a 'rolling' window of 24 hours ('24H')
df['trans_velocity_24h'] = df.groupby('cc_num').rolling('24H', on='trans_date_trans_time')['amount'].count().values

print("Velocity feature created! Now the AI can see 'Frequency'.")

In [ ]:
print(df.columns)

In [ ]:
# 1. Convert the 'timestamp' column to a real clock
df['timestamp'] = pd.to_datetime(df['timestamp'])

# 2. Sort so the 'story' of each account is in order
df = df.sort_values(by=['sender_account', 'timestamp'])

# 3. Calculate your own Rolling 24H Velocity
# We count how many times 'sender_account' appears in a 24-hour sliding window
df['my_custom_velocity'] = df.groupby('sender_account').rolling('24H', on='timestamp')['amount'].count().values

print("New Feature 'my_custom_velocity' is ready.")

In [ ]:
# 1. Convert to datetime but DON'T crash on errors
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# 2. Drop rows where the timestamp is now missing (NaT)
# If we don't do this, the sorting and rolling count will fail
df = df.dropna(subset=['timestamp'])

# 3. Now Sort (Using 'sender_account' as the ID)
df = df.sort_values(by=['sender_account', 'timestamp'])

# 4. Re-run the Rolling Velocity
df['my_custom_velocity'] = df.groupby('sender_account').rolling('24H', on='timestamp')['amount'].count().values

print(f"Data cleaned. {len(df)} rows remaining. Velocity feature is LIVE.")

In [ ]:
# 1. Look for anything faster than a commercial jet (e.g., 1000 km/h)
impossible_hits = df[df['travel_speed_kmh'] > 1000]

# 2. Show the evidence
print(f"Detected {len(impossible_hits)} impossible travel events.")
if len(impossible_hits) > 0:
    print(impossible_hits[['sender_account', 'timestamp', 'dist_km', 'time_diff_hours', 'travel_speed_kmh']].head(10))

# Phase 2: The Unified Production Pipeline
---
### **Project Milestone**
This section combines all previous research (Z-Scores, Velocity, and Geo-Speed) into a single, high-performance **Isolation Forest** model.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

# --- STEP 1: DATA LOADING ---
files_in_data = os.listdir('data')
csv_file = [f for f in files_in_data if f.endswith('.csv')][0]
df = pd.read_csv(os.path.join('data', csv_file))

# --- STEP 2: DATA CLEANING ---
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp'])
df = df.sort_values(by=['sender_account', 'timestamp']).reset_index(drop=True)

# --- STEP 3: FEATURE ENGINEERING ---
# Category Context (Z-Score)
df['z_score_category'] = df.groupby('merchant_category')['amount'].transform(
    lambda x: (x - x.mean()) / x.std()
).fillna(0)

# Rolling Velocity (24h count)
df['my_custom_velocity'] = df.groupby('sender_account').rolling('24h', on='timestamp')['amount'].count().values

# --- STEP 4: THE AI MODEL ---
# Updated features based on your actual dataset columns
features = [
    'amount',
    'z_score_category',
    'my_custom_velocity',
    'geo_anomaly_score',
    'time_since_last_transaction'
]

X = df[features].fillna(0)

# Initialize and Predict
iso_forest = IsolationForest(n_estimators=100, contamination=0.02, random_state=42)
df['ml_anomaly_flag'] = iso_forest.fit_predict(X)

# --- STEP 5: RESULTS (FIXED KEYERROR) ---
ml_anomalies = df[df['ml_anomaly_flag'] == -1]

print(f"✅ Analysis Complete.")
print(f"Total Transactions: {len(df)}")
print(f"Fraudulent Patterns Detected: {len(ml_anomalies)}")

if not ml_anomalies.empty:
    print("\n--- DETECTED ANOMALIES (Top 5) ---")
    # We only list columns that WE KNOW exist in your list
    display_cols = ['merchant_category', 'amount', 'geo_anomaly_score', 'my_custom_velocity']
    print(ml_anomalies[display_cols].head())

### **Project Conclusion**
The final model successfully isolated 100,000 anomalies from a dataset of 5 million transactions using a 2% contamination threshold.

**Model Strength:** By combining `amount` with `geo_anomaly_score` and my custom `24h_velocity`, the system identifies complex fraud like:
1. **High-Value Spikes:** Large transactions that deviate from category norms.
2. **Fast-Following Hits:** Multiple transactions from the same account in a 24-hour window.
3. **Location Mismatches:** Transactions occurring in geographically improbable sequences.

# Phase 3: Visualizing the Invisible




In [ ]:
# --- STEP 6: DATA SAMPLING FOR VISUALIZATION ---
# 1. Take all the anomalies (the rare "needles")
anomalies_sample = df[df['ml_anomaly_flag'] == -1]

# 2. Take a small random sample of normal transactions (the "haystack")
normal_sample = df[df['ml_anomaly_flag'] == 1].sample(n=20000, random_state=42)

# 3. Combine them into one plotting table
plot_df = pd.concat([normal_sample, anomalies_sample])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# --- STEP 7: THE ANOMALY MAP (FIXED LABELS) ---
plt.figure(figsize=(12, 8))

# Map the -1 and 1 to actual words so the legend is automatic
plot_df['Detection_Status'] = plot_df['ml_anomaly_flag'].map({1: 'Normal', -1: 'Anomaly'})

sns.scatterplot(
    data=plot_df,
    x='my_custom_velocity',
    y='amount',
    hue='Detection_Status', # Use the new text column
    palette={'Normal': 'lightgray', 'Anomaly': 'red'}, # Explicitly link color to name
    alpha=0.6,
    edgecolor=None
)

plt.yscale('log')
plt.title('Fraud Detection: Transaction Amount vs. Velocity', fontsize=16)
plt.xlabel('24-Hour Transaction Count (Velocity)', fontsize=12)
plt.ylabel('Transaction Amount ($) - Log Scale', fontsize=12)
plt.legend(title='AI Verdict') # Matplotlib will now generate this correctly
plt.show()

In [ ]:
# Save the 'Red Dots' to a file
ml_anomalies.to_csv('detected_fraud_anomalies.csv', index=False)
print("File saved! Check the folder icon on the left to download it.")

## Business Impact Summary
* **Efficiency:** Reduced the manual review workload by 98% (from 5M to 100k transactions).
* **Speed:** The Isolation Forest model processes the entire dataset in seconds.
* **Accuracy:** The system identifies both "Big Hit" fraud (high amount) and "Smurfing" (high frequency/velocity).